In [1]:

print("helloworld")



helloworld


In [1]:

import os
import findspark
findspark.init('/Users/jerdavis/devlib/spark-2.4.3-bin-hadoop2.7/')

os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /Users/jerdavis/devlib/xgboost/xgboost4j-spark-0.90.jar,/Users/jerdavis/devlib/xgboost/xgboost4j-0.90.jar,/Users/jerdavis/devhome/projects/pysparkgw/target/scala-2.11/pysparkgw_2.11-0.1.jar pyspark-shell'
findspark.init()


In [2]:


import numpy
import pyspark

from pyspark.sql.session import SparkSession
from pyspark.sql.types import *
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.sql.functions import col

spark = SparkSession.builder.appName("foo")\
    .master("local[*]")\
    .config("spark.driver.memory","8G")\
    .config("spark.driver.maxResultSize", "2G")\
    .getOrCreate()

    # .config("spark.driver.extraClassPath", "/Users/jerdavis/devlib/xgboost/xgboost4j-spark-0.90.jar:/Users/jerdavis/devlib/xgboost/xgboost4j-0.90.jar:/Users/jerdavis/devhome/projects/pysparkgw/target/scala-2.11/pysparkgw_2.11-0.1.jar")\
    # .config("spark.executor.extraClassPath", "/Users/jerdavis/devlib/xgboost/xgboost4j-spark-0.90.jar:/Users/jerdavis/devlib/xgboost/xgboost4j-0.90.jar:/Users/jerdavis/devhome/projects/pysparkgw/target/scala-2.11/pysparkgw_2.11-0.1.jar")\
    # .config("spark.jars", "/Users/jerdavis/devlib/xgboost/xgboost4j-spark-0.90.jar:/Users/jerdavis/devlib/xgboost/xgboost4j-0.90.jar:/Users/jerdavis/devhome/projects/pysparkgw/target/scala-2.11/pysparkgw_2.11-0.1.jar")\

#spark.sparkContext.addPyFile("/Users/jerdavis/devlib/xgboost/sparkxgb.zip")

#import sparkxgb
#from sparkxgb import XGBoostEstimator


# selfHelper = spark.sparkContext._jvm.jd.PyTest
# selfHelper.quoteRandallzz()



In [3]:

selfHelper = spark.sparkContext._jvm.jd.PyTest
#selfHelper.foo(123,"smoo")
print(selfHelper.bar(123,"smoo",spark.sparkContext._jsc))

rr = spark.sql("SELECT * from rr")
rr.show()

# schema = StructType(
#   [StructField("PassengerId", DoubleType()),
#     StructField("Survival", DoubleType()),
#     StructField("Pclass", DoubleType()),
#     StructField("Name", StringType()),
#     StructField("Sex", StringType()),
#     StructField("Age", DoubleType()),
#     StructField("SibSp", DoubleType()),
#     StructField("Parch", DoubleType()),
#     StructField("Ticket", StringType()),
#     StructField("Fare", DoubleType()),
#     StructField("Cabin", StringType()),
#     StructField("Embarked", StringType())
#   ])
# 
# df_raw = spark\
#   .read\
#   .option("header", "true")\
#   .schema(schema)\
#   .csv("/Users/jerdavis/devlib/xgboost/train.csv")
# df = df_raw.na.fill(0)



+-----------+----------+
|PassengerId|prediction|
+-----------+----------+
|        1.0|       0.0|
|        4.0|       1.0|
|       14.0|       0.0|
|       15.0|       1.0|
|       20.0|       1.0|
|       28.0|       1.0|
|       34.0|       0.0|
|       38.0|       0.0|
|       50.0|       1.0|
|       52.0|       0.0|
|       59.0|       1.0|
|       60.0|       0.0|
|       82.0|       0.0|
|       94.0|       0.0|
|       96.0|       0.0|
|       99.0|       1.0|
|      104.0|       0.0|
|      105.0|       0.0|
|      107.0|       1.0|
|      116.0|       0.0|
+-----------+----------+
only showing top 20 rows



In [10]:
sexIndexer = StringIndexer()\
  .setInputCol("Sex")\
  .setOutputCol("SexIndex")\
  .setHandleInvalid("keep")
    
cabinIndexer = StringIndexer()\
  .setInputCol("Cabin")\
  .setOutputCol("CabinIndex")\
  .setHandleInvalid("keep")
    
embarkedIndexer = StringIndexer()\
  .setInputCol("Embarked")\
  .setOutputCol("EmbarkedIndex")\
  .setHandleInvalid("keep")

vectorAssembler = VectorAssembler()\
  .setInputCols(["Pclass", "SexIndex", "Age", "SibSp", "Parch", "Fare", "CabinIndex", "EmbarkedIndex"])\
  .setOutputCol("features")

foo = {"featuresCol":"features","labelCol":"Survival","predictionCol":"prediction"}

#xgboost = XGBoostEstimator(foo)
    # featuresCol="features", 
    # labelCol="Survival", 
    # predictionCol="prediction",
    # maxDepth="20",


# xgboost = XGBoostEstimator(
#     featuresCol="features", 
#     labelCol="Survival", 
#     predictionCol="prediction")
    
 
pipeline = Pipeline().setStages([sexIndexer, cabinIndexer, embarkedIndexer, vectorAssembler, xgboost])



In [5]:
trainDF, testDF = df.randomSplit([0.8, 0.2], seed=24)
model = pipeline.fit(trainDF)
model.transform(testDF).select(col("PassengerId"), col("prediction")).show()



+-----------+----------+
|PassengerId|prediction|
+-----------+----------+
|        1.0|       0.0|
|        4.0|       1.0|
|       14.0|       0.0|
|       15.0|       1.0|
|       20.0|       1.0|
|       28.0|       1.0|
|       34.0|       0.0|
|       38.0|       0.0|
|       50.0|       1.0|
|       52.0|       0.0|
|       59.0|       1.0|
|       60.0|       0.0|
|       82.0|       0.0|
|       94.0|       0.0|
|       96.0|       0.0|
|       99.0|       1.0|
|      104.0|       0.0|
|      105.0|       0.0|
|      107.0|       1.0|
|      116.0|       0.0|
+-----------+----------+
only showing top 20 rows

